<a href="https://colab.research.google.com/github/AndreyKuratov/project_mldm_21/blob/main/MLDM_proj_5_NN_with_multilabel_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import soundfile as sf
import scipy.signal as signal
import os
from tqdm import tqdm

import os
import csv
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image

import librosa as lb 
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd

from skimage.transform import resize
from scipy import stats

from sklearn.preprocessing import MinMaxScaler
from skimage.transform import resize

import torchvision.models as models
import torch

import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import DataLoader

# 1.Загрузка файлов и подготовка к обучению

In [24]:
PATH_TO_DIR = '/content/drive/MyDrive/mldm21_project/'
PATH_TO_SAVE = '/content/drive/MyDrive/mldm21_project/'

data_tp = np.load(PATH_TO_SAVE+'sr_48e3_nfft_2e11_hlop_2e9_pict_256x512_u8_tp.npz',allow_pickle=True)
# 'imgs' 'labels'
data_fp = np.load(PATH_TO_SAVE+'sr_48e3_nfft_2e11_hlop_2e9_pict_256x512_u8_fp.npz',allow_pickle=True)
# 'imgs' 'labels'

data_majorTest = np.load(PATH_TO_SAVE+'sr_48e3_nfft_2e11_hlop_2e9_pict_256x512_u8_majorTest.npz',allow_pickle=True)
# 'imgs' 'labels'
labels = np.load(PATH_TO_SAVE+'labels_simple_v2.npz')
# 'files_fp' 'labels_fp' 'files_tp' 'labels_tp'

train_files = np.load('/content/drive/MyDrive/mldm21_project/train_ids_0.npy',allow_pickle=True) 
test_files = np.load('/content/drive/MyDrive/mldm21_project/test_ids_0.npy',allow_pickle=True) 

pd_fp = pd.read_csv('/content/drive/MyDrive/mldm21_project/train_fp.csv')
pd_tp = pd.read_csv('/content/drive/MyDrive/mldm21_project/train_tp.csv')

Создание обучающего и тестового набора данных

(1132, 24)

In [25]:
mask_test = np.isin(data_tp['labels'], test_files)
mask_train = ~np.isin(data_tp['labels'], test_files)

In [31]:
TRAIN_DATA_TP = data_tp['imgs'][mask_train]
TRAIN_LABEL_TP = labels['labels_tp'][mask_train]
TRAIN_DATA_FP = data_fp['imgs']
TRAIN_LABEL_FP = labels['labels_fp']

TRAIN_DATA = np.concatenate((TRAIN_DATA_TP,TRAIN_DATA_FP),axis=0)
TRAIN_LABEL = np.concatenate((TRAIN_LABEL_TP,TRAIN_LABEL_FP),axis=0).copy()

TEST_DATA = data_tp['imgs'][mask_test]
TEST_LABEL = labels['labels_tp'][mask_test].copy()

TRAIN_DATA = np.swapaxes(TRAIN_DATA,1,3).copy()
TEST_DATA = np.swapaxes(TEST_DATA,1,3).copy()


In [38]:
MAJOR_TEST = np.swapaxes(data_majorTest['imgs'],1,3).copy()
MAJOR_FILES = data_majorTest['labels']

Сохранение данных на будущее

In [39]:
PATH_TO_SAVE = '/content/drive/MyDrive/mldm21_project/'
np.savez(PATH_TO_SAVE+'pure_train',data=TRAIN_DATA,labels=TRAIN_LABEL)
np.savez(PATH_TO_SAVE+'pure_test',data=TEST_DATA,labels=TEST_LABEL)
np.savez(PATH_TO_SAVE+'MAJOR_test', data =MAJOR_TEST, files=MAJOR_FILES)

# 2. Функции для загрузки теста и трейна и датасеты

In [3]:
import torchvision.models as models
import torch

import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

from sklearn.metrics import label_ranking_average_precision_score

def crtclass(mdTp,num_classes=24):
  if mdTp == 'Res18':
    modelF = models.resnet18()
    modelF.fc = nn.Linear(512, num_classes)
  elif mdTp == 'Res34':
    modelF = models.resnet34()
    modelF.fc = nn.Linear(512, num_classes)
  elif mdTp == 'Res50':
    modelF = models.resnet50()
    modelF.fc = nn.Linear(2048, num_classes)
  elif mdTp == 'Alex':
    modelF = models.alexnet()
    modelF.classifier[6] = nn.Linear(4096,num_classes)
  elif mdTp == 'VGG':
    modelF = models.vgg16()
    modelF.classifier[6] = nn.Linear(4096,num_classes)
  elif mdTp == 'Squeez':
    modelF = models.squeezenet1_0()
    modelF.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
  elif mdTp == 'Modile_v3':
    modelF = models.mobilenet_v3_small()
    modelF.classifier[0] = nn.Linear(in_features=576, out_features=4096, bias=True)
    modelF.classifier[3] = nn.Linear(4096,num_classes,bias=True)
  else:
    assert False, 'no model'
  return modelF

def train_step(model,idata,ilabels,criterion,optimizer,DEVICE):
    model.zero_grad()
    idata = idata.type(torch.FloatTensor).to(DEVICE)
    ilabels = ilabels.type(torch.FloatTensor).to(DEVICE)
    train_output = model(idata)
    #optimizer.zero_grad()
    loss = criterion(train_output, ilabels)
    loss.backward()
    optimizer.step()
    return loss.item()

def test_acc(test_loader,model,DEVICE):
    def sigmoid(x):
      return 1 / (1 + np.exp(-x))

    model.eval()
    acc_array=[]
    labels_output = []
    labels_true = []
    with torch.no_grad():
      for i_test_data, i_test_labels in test_loader:
            i_test_data = i_test_data.type(torch.FloatTensor).to(DEVICE)
            i_test_labels = i_test_labels.detach().numpy()
            test_output = model(i_test_data)
            test_output = test_output.cpu().detach().numpy()

            labels_true+=list(i_test_labels)
            labels_output+=list(test_output)

            test_label = np.argmax(test_output,axis=1)
            true_label = np.argmax(i_test_labels,axis=1)
            acc_array.extend((test_label==true_label))
    labels_true = np.array(labels_true)
    labels_output = sigmoid(np.array(labels_output))
    try:
      score = label_ranking_average_precision_score(labels_true, labels_output)
    except:
      score = 0.0
    return (np.sum(acc_array)/len(acc_array)),score

class simpleDataset(Dataset):
    def __init__(self, numpy_dataset,numpy_labels,transform=None, target_transform=None):
        self.dataset = numpy_dataset
        self.labels = numpy_labels
        self.length = len(numpy_labels)
        self.target_transform = target_transform
        self.transform = transform

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if self.transform != None:
          data = self.transform(self.dataset[idx])
        else:
          data = self.dataset[idx]
        if self.target_transform != None:
          target = self.target_transform(self.labels[idx])
        else:
          target = self.labels[idx]
        return data, target

# 3. Обучение

In [4]:
TEST_SIZE = 0.1
BATCH_SIZE = 16

LR = 1e-1
LR_red = 1e-2

MOMENTUM = 0.9

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCH_NUM = 30

update_best_models_Flag = True
print(DEVICE)

cpu


### Загрузка данных

In [5]:
PATH_TO_DIR = '/content/drive/MyDrive/mldm21_project/'
PATH_TO_SAVE = '/content/drive/MyDrive/mldm21_project/'

data_tr = np.load(PATH_TO_SAVE+'pure_train.npz')
data_ts = np.load(PATH_TO_SAVE+'pure_test.npz')
data_mts = np.load(PATH_TO_SAVE+'MAJOR_test.npz')

train_data = data_tr['data']
train_labels = data_tr['labels']
test_data = data_ts['data']
test_labels = data_ts['labels']
majot_test_data = data_mts['data']

In [6]:
train_dataset = simpleDataset(train_data,train_labels)
test_dataset = simpleDataset(test_data,test_labels)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

### Создание сеток

In [7]:
import torch.optim as optim

model_18 = crtclass('Res18').to(DEVICE)
best_model_18 = crtclass('Res18')
criterion_18 = nn.BCEWithLogitsLoss()
optimizer_18 = optim.SGD(model_18.parameters(), lr=LR, momentum=MOMENTUM)

model_50 = crtclass('Res50').to(DEVICE)
best_model_50 = crtclass('Res50')
criterion_50 = nn.BCEWithLogitsLoss()
optimizer_50 = optim.SGD(model_50.parameters(), lr=LR, momentum=MOMENTUM)



models_arr = [model_18,model_50]

criterion_arr = [criterion_18,criterion_50]
optimizer_arr = [optimizer_18,optimizer_50]

num_models = len(models_arr)

In [8]:
best_models_arr = [best_model_18,best_model_50]
ModelNames = ['Res18','Res50']

In [10]:
import copy 
from IPython import display


loss_arr = []
test_acc_array = [ [] for i in range(num_models)]
test_score_array = [ [] for i in range(num_models)]
max_acc = [ 0 for i in range(num_models)]

for iepoch in tqdm(range(EPOCH_NUM)):
  inter_loss_arr = []
  for imodel in models_arr:
    imodel.train()
  for idata, ilabels in train_loader:
    cur_loss_arr = [ [] for i in range(num_models)]
    intI=0
    for imodel,icriterion ,ioptimizer in zip(models_arr,criterion_arr,optimizer_arr):
      cur_loss = train_step(imodel,idata,ilabels,icriterion,ioptimizer,DEVICE)
      cur_loss_arr[intI] = cur_loss
      intI+=1
    inter_loss_arr.append(cur_loss_arr)
  loss_arr.append(np.mean(inter_loss_arr,axis=0))
  cur_test_acc = []
  intI=0

  for imodel,ibest_model in zip(models_arr,best_models_arr):
    acc_cur,score_cur = test_acc(test_loader,imodel,DEVICE)
    test_acc_array[intI].append(acc_cur)
    test_score_array[intI].append(score_cur)
    cur_test_acc.append(acc_cur)
    if (acc_cur > max_acc[intI]) and (update_best_models_Flag):
      max_acc[intI]=acc_cur
      sd = imodel.state_dict()
      ibest_model.load_state_dict(sd)
    intI+=1

  strOUT_1 = 'eph.: {:} '.format(iepoch)
  strOUT_2 = 'loss:' + str(loss_arr[-1])
  strOUT_3 = 'eph.acc. : '+ str(cur_test_acc)
  strOUT_4 =  'best acc. : '+ str(max_acc)
  print(strOUT_1)
  print(strOUT_2)
  print(strOUT_3)
  print(strOUT_4)
    


  # plot figures in epoch
  loss_arr_np = np.array(loss_arr)
  test_acc_array_np = np.array(test_acc_array)
  test_score_array_np = np.array(test_score_array)

  plt.figure(figsize=(18,4))
  plt.subplot(1,3,1)
  plt.suptitle(strOUT_1+'   '+strOUT_2)
  plt.title('loss')
  plt.xlabel('Epoch')
  plt.ylabel('loss')
  for idx,inamne in enumerate(ModelNames):
    plt.plot(loss_arr_np[:,idx],label=inamne)
  plt.legend()

  plt.subplot(1,3,2)
  plt.title('acc. top-1')
  plt.xlabel('Epoch')
  plt.ylabel('test acc.')
  for idx,inamne in enumerate(ModelNames):
    plt.plot(test_acc_array_np[idx],label=inamne)
  plt.legend()

  plt.subplot(1,3,3)
  plt.title('score LRAP')
  plt.xlabel('Epoch')
  plt.ylabel('test score')
  for idx,inamne in enumerate(ModelNames):
    plt.plot(test_score_array_np[idx],label=inamne)
  plt.legend()

  display.clear_output(wait=True)
  display.display(plt.gcf())

  0%|          | 0/30 [01:26<?, ?it/s]


KeyboardInterrupt: ignored